In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [2]:

symbols = ["H", "H", "H"]


def h3_ground_energy(bond_length):
    
    """
    Uses VQE to calculate the ground energy of the H3+ molecule with the given bond length.
    
    Args:
        - bond_length(float): The bond length of the H3+ molecule modelled as an
        equilateral triangle.
    Returns:
        - Union[float, np.tensor, np.array]: A float-like output containing the ground 
        state of the H3+ molecule with the given bond length.
    """
    

    L=bond_length
    
    x = np.array([-L/2/np.sqrt(3), L/2, 0.0, L/np.sqrt(3), 0.0, 0.0,-L/2/np.sqrt(3), -L/2, 0.0])#/a0
    symbols = ["H","H","H"]
    
    Charge=1
    Hamiltonian=qml.qchem.molecular_hamiltonian( symbols, x,charge=1, basis="sto-3g")[0]
    

    num_qubits = len(Hamiltonian.wires)
    Electrun_num=3-Charge
    hf_state=qml.qchem.hf_state(electrons=Electrun_num, orbitals=num_qubits)

    singles, doubles = qml.qchem.excitations(Electrun_num, num_qubits)
    dev = qml.device("default.qubit", wires=num_qubits)
    @qml.qnode(dev)
    def cost(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
        
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
            
        """
        #state=np.array([0]*(num_qubits//2)+[1]*(num_qubits//2))
        qml.BasisState(hf_state, wires=list(range(num_qubits)))

        for i in range(len(singles)):
            #for j in range(E_len):
            qml.SingleExcitation(weights[i], wires=singles[i])

        for i in range(len(doubles)):
            qml.DoubleExcitation(weights[i+len(singles)], wires=doubles[i])
        
        return qml.expval(Hamiltonian)
    
    #np.random.seed = 1234
    weights = np.random.normal(
        0, np.pi, len(singles) + len(doubles), requires_grad=True
    )
    opt = qml.AdamOptimizer(0.5)

    for _ in range(100):
        weights = opt.step(cost, weights)

    return cost(weights)

# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    outs = h3_ground_energy(ins)
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(solution_output,expected_output, atol = 1e-4), "Not the correct ground energy"

In [3]:
# These are the public test cases
test_cases = [
    ('1.5', '-1.232574'),
    ('0.8', '-0.3770325')
]

In [4]:
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.5'...
Correct!
Running test case 1 with input '0.8'...
Correct!
